# Лабораторная работа 1. Извлечение признаков из текстовых данных


## Задание
- Реализовать алгоритм получения контекстных эмбеддингов (векторных представлений) слов.

- Реализовать один из алгоритмов CBoW или SkipGram для получения собственных моделей Word2Vec для выбранных данных.

- Визуализировать векторные представления в малой размерности.



## Ход работы
1. Загрузить коллекцию текстовых документов: просьбы жильцов. Можете использовать не весь датасет (подмножество строк таблицы). В этой ЛР вас интересует столбец с текстом обращений жильцов. Можете загрузить любой датасет для классификации текстовых данных.

2. Рассмотреть каждый текстовый документ отдельно. Выполнить предварительную обработку текстовых данных (как в ЛР 8 по ML). Токенизация, лемматизация (стемминг), удаление стоп-слов.

3. Составить матрицу контекстных эмбеддингов слов.

4. С помощью PCA уменьшить размер контекстных эмбеддингов (размер задать самостоятельно).

5. С помощью PyTorch реализовать один из алгоритмов Word2Vec: CBoW или SkipGram. Получить векторные представления текстов с помощью собственной реализации одного из этих алгоритмов.

6. Сохранить полученные различными способами векторные представления в файлы tsv. Визуализировать их с помощью сервиса Projector от авторов TensorFlow.

In [108]:
import pandas as pd
import torch
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
import re
from pymorphy3 import MorphAnalyzer
import string

### 1. Загрузить коллекцию текстовых документов: просьбы жильцов. Можете использовать не весь датасет (подмножество строк таблицы). В этой ЛР вас интересует столбец с текстом обращений жильцов. Можете загрузить любой датасет для классификации текстовых данных.

In [109]:
data = pd.read_csv("../data/Petitions.csv")

In [110]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59889 entries, 0 to 59888
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    59889 non-null  int64 
 1   public_petition_text  59889 non-null  object
 2   reason_category       59889 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [111]:
data.head()

,id,public_petition_text,reason_category
0,3168490,снег на дороге,Благоустройство
1,3219678,очистить кабельный киоск от рекламы,Благоустройство
2,2963920,"Просим убрать все деревья и кустарники, которы...",Благоустройство
3,3374910,Неудовлетворительное состояние парадной - надп...,Содержание МКД
4,3336285,Граффити,Благоустройство


In [112]:
data = data.head(10000)

In [113]:
data.drop("id", axis=1, inplace=True)

### 2. Рассмотреть каждый текстовый документ отдельно. Выполнить предварительную обработку текстовых данных (как в ЛР 8 по ML). Токенизация, лемматизация (стемминг), удаление стоп-слов.

Удаление всех символов, которые не являются цифрами, буквами и пробелами, и токенизация

In [114]:
morph = MorphAnalyzer()
stopwords = stopwords.words("russian")


def preProccesingData(text):
    only_letters = re.sub("[^\s^\w]+", "", text)
    tokenized = word_tokenize(only_letters)
    normalized = [morph.normal_forms(word)[0] for word in tokenized]
    no_stop_words = [
        word
        for word in normalized
        if ((word not in string.punctuation) or (word not in stopwords))
    ]
    return no_stop_words

In [115]:
data["public_petition_text"] = data["public_petition_text"].apply(preProccesingData)
data.head()

,public_petition_text,reason_category
0,"[снег, на, дорога]",Благоустройство
1,"[очистить, кабельный, киоск, от, реклама]",Благоустройство
2,"[просить, убрать, всё, дерево, и, кустарник, к...",Благоустройство
3,"[неудовлетворительный, состояние, парадный, на...",Содержание МКД
4,[граффити],Благоустройство
